This example is the most basic example of the usage of the package.
Here, we simply demonstrate its usage by using some functions of the package.

First, we import some util modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import Smoother
import PeakSearcher
import AreaCalculator
import BasicOperator
import OtherOperator

from Operator import Pipe, PipeNet
from Spectrum import Spectrum, SimulatedSpectrum

We generate a simulated spectrum.

In [ ]:
simu = Spectrum.SimulatedSpectrum()

We define some basic operators.

In [ ]:
cen = Smoother.CentroidSmoother(2)
strip = OtherOperator.SNIPStripper(4)
gauss = PeakSearcher.GaussPeakSearcher(2, 0.2)
wavelet = Smoother.WaveletSmoother('gauss', 'low', 6)
cal = AreaCalculator.AverageClassicPeakAreaCalculator()

In [ ]:



# strip test


sav = Smoother.SavitzkySmoother(3, 4)
strp = OtherOperator.SNIPStripper(5)

lin = BasicOperator.FunctionalOperator(func = lambda x: x)
dualstrp = BasicOperator.Stripper() 

baseline = Pipe([sav, strp])

schemes = [{'pipe': baseline, 'inp_id': 0, 'inp_order':[0], 'outp_id': 1, 'outp_order': 1},
        {'pipe': lin, 'inp_id': 0, 'inp_order':[0], 'outp_id': 1, 'outp_order': 0},
        {'pipe': Pipe([dualstrp, sav, sav]), 'inp_id': 1, 'inp_order':[0, 1], 'outp_id': 2, 'outp_order': 0},]
net = Operator.PipeNet(schemes)

net.plot()
net({0:simu})

spec = net.get_node(2)[0]
spec.plot()

simu.plot()
plt.savefig("fig.png")
